In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import random
from sklearn.model_selection import train_test_split
lemmataizer = WordNetLemmatizer()
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [30]:
def load_data(fileName = 'train.csv'):
    csv_path = os.path.join("../input",fileName)
    return pd.read_csv(csv_path)

In [31]:
df = load_data()

In [32]:
df.head()

In [33]:
df.info()

In [34]:
trainData, testData = train_test_split(df, test_size = 0.3, random_state=42)

In [35]:
def get_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [36]:
def getWordsByWriter(writer):
    filteredData = trainData[(trainData.author == writer)]
    wordDict = dict()
    stopwords = nltk.corpus.stopwords.words('english')
    for row in filteredData.itertuples():
        line = row.text.lower()        
        words = re.findall(r'\w+', line)
        tagWords = nltk.pos_tag(words)
        for word in tagWords:
            if word[0] in stopwords: 
                continue
            wordPos = get_pos(word[1])
            if wordPos == '':
                word = lemmataizer.lemmatize(word[0])
            else:
                word = lemmataizer.lemmatize(word[0], wordPos)
            try:
               wordDict[word] += 1
            except KeyError:
                wordDict[word] = 1
    return wordDict

In [37]:
import operator
eapDict = getWordsByWriter('EAP')
eapSortDict = sorted(eapDict.items(), key=operator.itemgetter(1), reverse=True)
print(eapSortDict)

#eapSortDict

In [39]:
hplDict = getWordsByWriter('HPL')
hplSortDict = sorted(hplDict.items(), key=operator.itemgetter(1), reverse=True)
#hplSortDict

In [38]:
mwsDict = getWordsByWriter('MWS')
mwsSortDict = sorted(mwsDict.items(), key=operator.itemgetter(1), reverse=True)
#mwsSortDict

In [40]:
print(len(eapSortDict))
print(len(hplSortDict))
print(len(mwsSortDict))

In [41]:
eap_64 = b'...'

In [42]:
hpl_64 = b'...'

In [43]:
mws_64 = b'...'

In [44]:
def createstrfromlist(writerList):
    writerStr = ''
    for entry in writerList:
        writerStr = writerStr + ' ' + entry[0] 
    return(writerStr)

In [45]:
import base64
import codecs
import imageio
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
def createWordCloud(writerPng, writer64, writerList, writerTitle):
    # Generate the Mask for EAP
    f1 = open(writerPng, "wb")
    f1.write(codecs.decode(writer64,'base64'))
    f1.close()
    img1 = imageio.imread(writerPng)
    # img = img.resize((980,1080))
    hcmask = img1
    plt.figure(figsize=(20,18))
    # The wordcloud of the raven for Edgar Allen Poe
    plt.subplot(211)
    wc = WordCloud(background_color="black", max_words=10000, 
                   mask=hcmask, stopwords=STOPWORDS, max_font_size= 40)
    wc.generate(createstrfromlist(writerList))
    plt.title(writerTitle)
    plt.imshow(wc.recolor( colormap= 'viridis' , random_state=17), alpha=0.9)
    plt.axis('off') 

In [46]:
createWordCloud("eap.png", eap_64, eapSortDict, "Edgar Allen Poe (The Raven)")
createWordCloud("hpl.png", hpl_64, hplSortDict, "HP Lovecraft (Cthulhu-Squidy)")
createWordCloud("mws.png", mws_64, mwsSortDict, "Mary Shelley (Frankenstein's Monster)")

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
sentence = ["I love to eat Burgers", 
            "I love to eat Fries"]
vectorizer = CountVectorizer(min_df=0)
sentence_transform = vectorizer.fit_transform(sentence)
print (sentence_transform)
print("The features are:\n {}".format(vectorizer.get_feature_names()))
print("\nThe vectorized array looks like:\n {}".format(sentence_transform.toarray()))

In [ ]:
def countAvgWordsInLine(writer):
    filteredData = trainData[(trainData.author == writer)]
    numberOfWords = 0
    numberOfLines = len(filteredData)
    for row in filteredData.itertuples():
        line = row.text.replace("'",'')
        words = re.findall(r'\w+', line)
        numberOfWords += len(words)
    print('total words ' + str(numberOfWords))
    print('total sentences ' + str(numberOfLines))
    return numberOfWords/numberOfLines

In [ ]:
print('avg of words per sentence ' + str(countAvgWordsInLine("EAP")))
print('avg of words per sentence ' + str(countAvgWordsInLine("HPL")))
print('avg of words per sentence ' + str(countAvgWordsInLine("MWS")))

In [ ]:
#LIST WITH ALL WORDS FROM ALL AUTHORS
finalWordList = {word for word,count in eapDict.items() if count > 10 }
finalWordList = {word for word, count in hplDict.items() if (count > 10) and (word not in finalWordList)}
finalWordList = {word for word, count in mwsDict.items() if (count > 10) and (word not in finalWordList)}

In [ ]:
for word in finalWordList:
    if word not in eapDict:
        eapDict[word] = 0
    if word not in hplDict:
        hplDict[word] = 0
    if word not in mwsDict:
        mwsDict[word] = 0

In [ ]:
def countWordInTagWords(tagWords):
    for word in tagWords:
        if word[0] in stopwords:
            continue
        wordPos = get_pos(word[1])
        if wordPos == '':
            word = lemmataizer.lemmatize(word[0])
        else:
            word = lemmataizer.lemmatize(word[0], wordPos)
        try:
            ansDict[word] += 1
        except KeyError:
            ansDict[word] = 1
    return ansDict

In [26]:
errorCount = 0
for row in testData.itertuples():
    ansDict = dict()
    line = row.text.lower()        
    words = re.findall(r'\w+', line)
    tagWords = nltk.pos_tag(words)
    ansDict = countWordInTagWords(tagWords)
    finalAuth = dict();
    finalAuth['EAP'] = 1
    finalAuth['HPL'] = 1
    finalAuth['MWS'] = 1
    for word in ansDict:
        countAuthWord = dict()
        countAuthWord['EAP'] = 0
        countAuthWord['HPL'] = 0
        countAuthWord['MWS'] = 0
        if word in eapDict:
            countAuthWord['EAP'] = eapDict[word]
        if word in hplDict:
            countAuthWord['HPL'] = hplDict[word]
        if word in mwsDict:
            countAuthWord['MWS'] = mwsDict[word]
        totalUse = sum(countAuthWord.values())
        for i in ['EAP','HPL','MWS']:
            if totalUse > 0 and countAuthWord[i] > 0:
                finalAuth[i] *= (countAuthWord[i] / totalUse)
                #print(i + " , author count: " + str(countAuthWord[i])+ " total count: "  + str(totalUse) + " preccent: " + str(finalAuth[i]))
    finalAuth = sorted(finalAuth.items(), key=operator.itemgetter(1), reverse=True)
    Auth = finalAuth[0][0]
    if Auth != row.author:
        #print("guessed: " + Auth + " truth: " + row.author)
        errorCount += 1
testSize = testData.count(axis=0, level=None, numeric_only=False).id
print("number of errors" + str(errorCount))
print("error is: " + str(errorCount/testSize))
